<a href="https://colab.research.google.com/github/SubashKhatiwada/RSA-vs-ECC/blob/main/ECC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import math
import random

#Global Parameters Generation 

In [72]:
a = 0
b = -4
p = 257 
G = [2,2]

# Find Base Point G in Ep(a,b)

In [73]:
# # Total points on elliptic curve under modulo p# 
# def total_points(a:int, b:int, p:int):
#   points_set = []
#   for  x in range(0,p-1):
#     for y in range(0,p-1):
#       if pow(y,2,p) == pow((pow(x,3)+a*x+b),1,p):
#         points_set.append([x,y])
#   return points_set

# def generate_base_point(points:list):
#   for i  in range(0,len(points)):
#     for j in range(0,len(points)):
#       print(j,"P of", points[i],"=",point_multiplication(j,points[i]))


# points = total_points(a,b,p)
# print(points)
# generate_base_point(points)

# Elliptic Curve Operations

In [75]:
# Extended Euclidean Algorithm for calculating Multiplicative Inverse#
def euclides(inverso, modulo):
    modulo0 = modulo
    x0, x1, y0, y1 = 1, 0, 0, 1
    while modulo != 0:
        q, inverso, modulo = inverso // modulo, modulo, inverso % modulo
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1    
    if x0 < 0:
        x0 += modulo0
    return x0

def curve_operation(P:list,Q:list):
  if P == Q:
    R = point_doubling(P)
  elif P[0] == Q[0] & P[1] == -Q[1] % p:
    R = 0
  else: 
    R = point_addition(P,Q)
  return R

def point_addition(P:list,Q:list):
  x1, y1, x2, y2 = P[0], P[1], Q[0], Q[1] 
  numerator = pow((y2-y1),1,p)
  denominator = pow((x2-x1),1,p)
  inverse_denominator = euclides(denominator,p)
  lamda = pow(numerator * inverse_denominator,1,p)
  x3 = pow(pow(lamda,2,p)-x1-x2,1,p)
  y3 = pow(lamda*(x1-x3)-y1,1,p)
  R = [x3,y3]
  return R

def point_doubling(P:list):
  x1, y1 = P[0], P[1]
  numerator = pow(3*pow(x1,2)+a,1,p)
  denominator = pow(2*y1,1,p)
  inverse_denominator = euclides(denominator,p)
  lamda = pow(numerator * inverse_denominator,1,p)
  x3 = pow(pow(lamda,2)-2*x1,1,p)
  y3 = pow(lamda*(x1-x3)-y1,1,p)
  R = [x3,y3]
  return R

def point_multiplication(k:int,P:list):
  R=P
  for i in range(2,k+1):
    R = curve_operation(R,P)
    if R==0:
      break
  return R

# print(point_multiplication(1,[283,315]))
  


# Key Generation for Alice

In [77]:
alice_private_key = 121
alice_public_key = point_multiplication(alice_private_key,G)
# print(alice_public_key)

# Key Generation for Bob

In [78]:
bob_private_key = 101
bob_public_key = point_multiplication(bob_private_key,G)
print(bob_public_key)

[197, 167]


# Shared Secret Key Generation

In [80]:
alice_secret_key = point_multiplication(alice_private_key, bob_public_key)
bob_secret_key = point_multiplication(bob_private_key, alice_public_key)
print(alice_secret_key)
print(bob_secret_key)

[192, 238]
[192, 238]


# Message Encoder and Decoder

In [180]:
def generate_table(G:list):
  table = []
  for i in range(0,127):
    point = point_multiplication(i,G)
    table.append([i,point])
  return table
encoder_decoder_table = generate_table(G)
print("encoder_decoder_table ",encoder_decoder_table)

def encoder(text:str):
  ascii_list = [ord(c) for c in text]
  encoded_points = []
  # print(ascii_list)
  for i in range(0,len(ascii_list)):
    encoded_point = encoder_decoder_table[ascii_list[i]][1]
    encoded_points.append(encoded_point)
  return encoded_points

def decoder(encoded_points:list):
  points_list = []
  ascii_list = []
  for j in range(127):
    value = encoder_decoder_table[j][1]
    points_list.append(value)
  indices = []
  for i in range(len(encoded_points)):
    index = points_list.index(encoded_points[i])
    ascii_list.append(index)
  message = ''.join(chr(i) for i in ascii_list)
  return(message)

# print("Encodeing=",encoder("hiii"))
# print("decoding",decoder([[178, 59], [99, 190], [99, 190], [99, 190]]))

encoder_decoder_table  [[0, [2, 2]], [1, [2, 2]], [2, [5, 246]], [3, [126, 107]], [4, [188, 226]], [5, [56, 248]], [6, [4, 46]], [7, [221, 63]], [8, [156, 137]], [9, [63, 53]], [10, [163, 251]], [11, [233, 43]], [12, [53, 29]], [13, [68, 84]], [14, [164, 7]], [15, [49, 14]], [16, [86, 157]], [17, [80, 56]], [18, [177, 35]], [19, [159, 127]], [20, [210, 1]], [21, [229, 114]], [22, [185, 13]], [23, [39, 132]], [24, [99, 67]], [25, [178, 198]], [26, [42, 117]], [27, [153, 110]], [28, [197, 90]], [29, [125, 156]], [30, [143, 91]], [31, [240, 123]], [32, [94, 124]], [33, [253, 73]], [34, [192, 19]], [35, [29, 235]], [36, [249, 189]], [37, [55, 141]], [38, [226, 70]], [39, [168, 122]], [40, [15, 162]], [41, [136, 128]], [42, [30, 221]], [43, [0, 32]], [44, [223, 229]], [45, [254, 45]], [46, [3, 58]], [47, [47, 48]], [48, [90, 245]], [49, [201, 164]], [50, [100, 254]], [51, [20, 172]], [52, [137, 8]], [53, [93, 171]], [54, [113, 159]], [55, [75, 205]], [56, [122, 41]], [57, [34, 39]], [58, [2

# ECC Message Encryption by Alice




In [149]:
def encryption(message:str, bob_public_key:list):
  Pm = encoder(message)
  # Pm = [[112,26],[113, 26]]
  print("message=",Pm)
  Pm_plus_kPB_list = []
  k = 41
  kG = point_multiplication(k,G)
  kPb = point_multiplication(k,bob_public_key)
  for i in range(0,len(Pm)):
    Pm_plus_kPB = point_addition(Pm[i],kPb)
    Pm_plus_kPB_list.append(Pm_plus_kPB)
  cipher = [kG,Pm_plus_kPB_list]
  print("cipher= ",cipher)
encryption("hii", bob_public_key)

message= [[178, 59], [99, 190], [99, 190]]
cipher=  [[136, 128], [[53, 228], [233, 214], [233, 214]]]


# ECC Chipher Decryption by Bob 

In [151]:
def decryption(cipher:list):
  kG = cipher[0]
  print("kG = ", kG)
  print("bob_private_key = ", bob_private_key)
  Pm_plus_kPB_list = cipher[1]
  nBkG = point_multiplication(bob_private_key,kG)
  print("nBkG = ",nBkG)
  negative_nBkG = [nBkG[0], pow(-nBkG[1],1,p)]
  print("-nBkG = ", negative_nBkG)
  Pm = []
  for i in range(0,len(Pm_plus_kPB_list)):
    decry_p = curve_operation(Pm_plus_kPB_list[i],negative_nBkG)
    Pm.append(decry_p)
  print(Pm)
  # message = decoder(Pm)

decryption([[136, 128], [[53, 228], [233, 214], [233, 214]]])

kG =  [136, 128]
bob_private_key =  101
nBkG =  [68, 84]
-nBkG =  [68, 173]
[[178, 59], [99, 190], [99, 190]]
